In [6]:
def apply_async(func, args, *, callback):
    result = func(*args)  # Compute the result
    callback(result)       # Invoke the callback

In [7]:
class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args

In [8]:
from queue import Queue
from functools import wraps

def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)          # Start the generator
        result_queue = Queue()   # Create a result queue
        result_queue.put(None)   # Prime the queue with an initial value
        while True:
            result = result_queue.get()  # Get the latest result
            try:
                a = f.send(result)       # Send the result to the generator
                apply_async(a.func, a.args, callback=result_queue.put)  # Schedule the next task
            except StopIteration:        # Exit when the generator is done
                break
    return wrapper

In [5]:
def add(x, y):
    return x + y

@inlined_async
def test():
    r = yield Async(add, (2, 3))          # First async task
    print(r)
    r = yield Async(add, ('hello', 'world'))  # Second async task
    print(r)
    for n in range(10):                   # Loop with async tasks
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')

In [9]:
if __name__ == '__main__':
    import multiprocessing
    pool = multiprocessing.Pool()
    apply_async = pool.apply_async  # Use multiprocessing's apply_async
    test()  # Run the test function

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye
